In [174]:
import pandas as pd
import sklearn

In [ ]:
##This parts reads in the files and joins them

In [175]:
engagements_tc = pd.read_csv('engagements_tc.txt',delimiter="\t")

In [176]:
print engagements_tc.columns

Index([u'post_id', u'events', u'engagement_date'], dtype='object')


In [178]:
meta_tc = pd.read_csv('meta_tc.txt',delimiter='\t+')

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


meta_tc.columns

In [181]:
meta_tc.set_index('post_id',inplace=True)
engagements_tc.set_index('post_id',inplace=True)

In [182]:
engagements_meta = pd.merge(meta_tc,engagements_tc,how='inner',left_index=True,right_index=True)

In [183]:
engagements_meta.columns

Index([u'link', u'title', u'summary', u'author_actor_id', u'pub_date',
       u'events', u'engagement_date'],
      dtype='object')

In [184]:
engagements_meta['engagement_date'] = engagements_meta['engagement_date'].astype(float)

In [185]:
engagements_meta.dtypes

link                object
title               object
summary             object
author_actor_id    float64
pub_date            object
events               int64
engagement_date    float64
dtype: object

In [186]:
import time
from datetime import datetime

In [187]:
def convertEngagementDate(x):
    x[6] = datetime.fromtimestamp(x[6]).strftime('%Y-%m-%d %H:%M:%S')
    #x[6] = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x[6]))
    return x

In [188]:
engagements_meta = engagements_meta.apply(convertEngagementDate,axis=1)

In [189]:
engagements_meta.head(1)

,link,title,summary,author_actor_id,pub_date,events,engagement_date
post_id,,,,,,,
676211298,http://techcrunch.com/2015/07/01/setback-for-e...,Setback For European Facebook Privacy Class Ac...,An ongoing Facebook class action suit in Europ...,6796.0,2015-07-01 04:00:00,560,2015-07-01 00:00:00


In [190]:
def convertPubDate(x):
    x[4] = datetime.strptime(x[4], "%Y-%m-%d %H:%M:%S")
    return x

In [191]:
engagements_meta.dtypes

link                object
title               object
summary             object
author_actor_id    float64
pub_date            object
events               int64
engagement_date     object
dtype: object

In [192]:
engagements_meta = engagements_meta.apply(convertPubDate,axis=1)

In [193]:
def dateDifference(x): 
    x[6] = datetime.strptime(str(x[6]), "%Y-%m-%d %H:%M:%S")
    x[7] = abs((x[6] - x[4]).days)
    return x
engagements_meta.dtypes

link                       object
title                      object
summary                    object
author_actor_id           float64
pub_date           datetime64[ns]
events                      int64
engagement_date            object
dtype: object

In [ ]:
# PART 2
# Feature Extraction
# Since we have to find views for the first day we find the events for the first day
# Do analysis on link url - Analysis Results : All the events came from one view
# Find AuthorEventsRation = Views by Author/ Total number of views
# LDA to find topics for the authors - Could not be completed because of issues with one hot encoding. If resolved could have
# been used as feature

In [385]:
engagements_meta['date_difference'] = 1
engagements_meta = engagements_meta.apply(dateDifference,axis=1)

ValueError: ("time data '1' does not match format '%Y-%m-%d %H:%M:%S'", u'occurred at index 6796.0')

In [196]:
authorEventCount = engagements_meta.groupby('author_actor_id')['events'].sum()

In [201]:
authorEventCount = pd.DataFrame(authorEventCount)
authorEventCount.columns = ['Total_events']
authorEventCount.head(1)
totalEventCount = authorEventCount['Total_events'].sum()
authorEventCount.columns = ['AuthorEventsRatio']

In [202]:
def calculateEventCountRatio(x):
    x = (x*1.0)/(totalEventCount*1.0)
    return x

In [203]:
authorEventCount = authorEventCount.applymap(lambda x : calculateEventCountRatio(x))

In [229]:
authorEventCount.head(1)
authorEventCount.to_csv('AuthorEventsRatio.csv')

In [205]:
engagements_meta['post_id'] = engagements_meta.index
engagements_meta.set_index('author_actor_id',inplace=True)
engagements_meta_author_ratio = pd.merge(authorEventCount,engagements_meta,left_index=True,right_index=True, how="inner")

In [206]:
engagements_meta_author_ratio.columns

Index([u'AuthorEventsRatio', u'link', u'title', u'summary', u'pub_date',
       u'events', u'engagement_date', u'date_difference', u'post_id'],
      dtype='object')

In [207]:
engagements_meta_author_ratio = engagements_meta_author_ratio.sort(['AuthorEventsRatio','events'],ascending=[0,0])

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [ ]:
#
# Link Analysis - Did not give sufficient value - Events from only one url
#

In [208]:
linkGrouping = engagements_meta_author_ratio['link']
linkGrouping = pd.DataFrame(linkGrouping)
linkGrouping.columns

Index([u'link'], dtype='object')

In [209]:
from urlparse import urlparse
def extractLinkUrl(x):
    parsed_uri = urlparse(x)
    domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    return domain

In [210]:
linkGrouping = linkGrouping.applymap(lambda x: extractLinkUrl(x))
linkGrouping.drop_duplicates(['link'])

,link
author_actor_id,
529.0,http://techcrunch.com/
3869929.0,http://www.techcrunch.com/
8306862.0,http://social.techcrunch.com/


In [ ]:
#
# START LDA WORK
# Could not be completed because of problems with one hot encoding
#

In [211]:
summary = engagements_meta_author_ratio[['summary','post_id']]

summary.head(1)

In [212]:
summary = pd.DataFrame(summary)

In [213]:
summary.head(5)

,summary,post_id
author_actor_id,,
529.0,A startup called VINA is launching a mobile ap...,749630465
529.0,At New York's Toy Fair trade show over the wee...,753895300
529.0,Amazon is always looking for ways to boost the...,720754749
529.0,At New York's Toy Fair trade show over the wee...,753565585
529.0,Microsoft's newly launched A.I.-powered bot ca...,770116013


In [214]:
summary['author_actor_id'] = summary.index

In [215]:
summaryList = summary.values.tolist()

summaryList[1]

In [216]:
print summaryList[4]

['Microsoft\'s newly launched A.I.-powered bot called Tay, which was responding to tweets and chats on GroupMe and Kik, has already been shut down due to concerns with its inability to recognize when it was making offensive or racist statements. Of course, the bot wasn\'t coded to be racist, but it "learns" from those it interacts with. And...', 770116013, 529.0]


In [217]:
AuthorTextIndex = []
def combineText():
    startIndex = int(summaryList[1][1])
    text = ''
    for x in summaryList:
        if int(x[1]) == startIndex:
            text = text + x[0]
        elif int(x[1]) != startIndex:
            AuthorTextIndex.append((startIndex,text))
            startIndex = int(x[1])
            text = x[0]

combineText()          
            

In [218]:
len(AuthorTextIndex)

26793

In [231]:
AuthorTextIndex[2]

(753895300,
 'At New York\'s Toy Fair trade show over the weekend, Mattel unveiled its new, $300 3D Printer, the "ThingMaker," which will allow children to print their own toys at home. The device works in conjunction with a 3D printing app developed in collaboration with Autodesk that offers a simple interface for designing items that can then come to life...')

In [220]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
vectorizer = TfidfVectorizer(min_df=1,stop_words='english',ngram_range=(2, 2))

In [221]:
AuthorText = [x[1] for x in  AuthorTextIndex]

In [222]:
print len(AuthorText)

26793


In [223]:
AuthorTextVectorizer = vectorizer.fit_transform(AuthorText)
idf = vectorizer.idf_
idf.sort

<function sort>

In [224]:
from sklearn.decomposition import NMF, LatentDirichletAllocation


In [225]:
#nmf = NMF(n_components=10, random_state=1, alpha=.1, l1_ratio=.5).fit(AuthorTextVectorizer)
topicList = []
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        s = " ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])
        topicList.append(s)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [251]:
lda = LatentDirichletAllocation(n_topics=1000, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0)

In [252]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(AuthorText)

In [253]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=1000, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [254]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names,5)


Topics in LDA model:
Topic #0:
economic spent testing helps women
Topic #1:
remember range check sf says
Topic #2:
july late payment man marketplace
Topic #3:
short plus got china light
Topic #4:
airbnb bank experience location revealed
Topic #5:
zuckerberg yahoo windows continue ebay
Topic #6:
hardware reality revenue likely addition
Topic #7:
coming tools square offer step
Topic #8:
looks smartphone iphone opened analytics
Topic #9:
network social access local job
Topic #10:
true iot prime women bank
Topic #11:
reality credit google legal people
Topic #12:
terms exactly investments million healthcare
Topic #13:
audience point intelligence add entertainment
Topic #14:
million developed silicon ride simply
Topic #15:
term left resources history annual
Topic #16:
service cloud company management new
Topic #17:
hours consumers help developers pretty
Topic #18:
roll revealed devices particularly plenty
Topic #19:
reddit week easier fintech sure
Topic #20:
check accounts videos bring job


TypeError: dump() takes at least 2 arguments (1 given)

In [255]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
label_encoder_list = " ".join(topicList)
label_encoder_list = label_encoder_list.split(' ')
len(label_encoder_list)

5500

In [256]:
le.fit(label_encoder_list)
label_encoder_set = set(label_encoder_list)
label_encoder_df = pd.DataFrame(list(label_encoder_set))
label_encoder_df.to_csv('label_encoder.csv')

In [297]:
AuthorTextCopy = [unicode(x[0],"utf-8") for x in summaryList]
summaryReducedTopics = []
for x in AuthorTextCopy:
    tempList = x.split(" ")
    topicReduced = " ".join([str(val) for val in tempList if val in label_encoder_list])
    if topicReduced != '':
        summaryReducedTopics.append(topicReduced)

In [269]:
from sklearn.externals import joblib
joblib.dump(le, 'lda.pkl')

['lda.pkl', 'lda.pkl_01.npy']

In [268]:
le.transform(summaryReducedTopics)

ValueError: y contains new labels: [ '10 doing announcing operating 200 million monthly active devices company today used announce number million monthly active'
 '10 early long support specific company announced developers conference expanding support operating developers'
 '10 good week sports live stream options making service' ...,
 'young children old remember car want founders campaign'
 'young man named soon want person right meet looking companies pitch like tell world'
 'young startup founders startup ways half period']

In [270]:
engagements_meta_author_ratio.head(1)

,AuthorEventsRatio,link,title,summary,pub_date,events,engagement_date,date_difference,post_id
author_actor_id,,,,,,,,,
529.0,0.068907,http://techcrunch.com/2016/01/26/hey-vina-is-a...,"""Hey! VINA"" Is A Tinder For (Girl) Friends",A startup called VINA is launching a mobile ap...,2016-01-26 05:00:00,25231,2016-01-01,26,749630465


In [282]:
datasetDf = engagements_meta_author_ratio[(engagements_meta_author_ratio['date_difference'] <= 1)]

In [283]:
datasetDf

,AuthorEventsRatio,link,title,summary,pub_date,events,engagement_date,date_difference,post_id
author_actor_id,,,,,,,,,
529.0,0.068907,http://techcrunch.com/2015/09/01/paypal-launch...,"PayPal Launches PayPal.Me, A Simpler Way To Re...",PayPal today is launching a new take on peer-t...,2015-09-01 04:00:00,3253,2015-09-01,1,710857149
529.0,0.068907,http://techcrunch.com/2015/12/01/google-turns-...,Google Turns Image Search Into Pinterest With ...,"Watch out, Pinterest, you've got new competiti...",2015-12-01 05:00:00,2528,2015-12-01,1,739701762
529.0,0.068907,http://techcrunch.com/2016/02/29/google-maps-b...,"Google Maps Brings Its ""Add A Pit Stop"" Featur...","Last fall, Google announced the addition of a ...",2016-02-29 05:00:00,2351,2016-03-01,0,755950398
529.0,0.068907,http://techcrunch.com/2015/10/01/amazon-bans-s...,Amazon Bans Sales Of Apple TV And Chromecast O...,In a anti-competitive move that bumps up again...,2015-09-30 04:00:00,2218,2015-10-01,0,723779873
529.0,0.068907,http://techcrunch.com/2016/03/31/amazon-expand...,"Amazon expands Dash Button line-up, top seller...",Amazon this morning announced an expansion of ...,2016-03-31 04:00:00,1882,2016-04-01,0,771932109
529.0,0.068907,http://techcrunch.com/2015/08/31/keyraider-mal...,KeyRaider Malware Responsible For Possibly Lar...,Jailbreaking your iPhone has its downsides. In...,2015-08-31 04:00:00,1636,2015-09-01,0,710639095
529.0,0.068907,http://techcrunch.com/2015/10/02/a-new-app-cal...,A New App Called Soon Organizes Your Everyday ...,Your friend tells you about a great new TV sho...,2015-10-01 04:00:00,1631,2015-10-01,1,724666386
529.0,0.068907,http://techcrunch.com/2016/02/29/facebooks-pri...,Facebook's Private Photo-Sharing App Moments N...,"Facebook's Moments, the social network's priva...",2016-02-29 05:00:00,1606,2016-03-01,0,755949265
529.0,0.068907,http://techcrunch.com/2016/03/02/whatsapp-adds...,"WhatsApp adds support for document sharing, bu...","WhatsApp, the Facebook-owned mobile messaging ...",2016-03-01 05:00:00,1503,2016-03-01,1,756897316


In [ ]:
#
# BUILDING THE REGRESSION MODEL
# Using SVR with Rbf as final model
#

In [284]:
datasetDf = datasetDf[['AuthorEventsRatio','summary','date_difference','events','post_id']]

In [327]:
from sklearn.preprocessing import OneHotEncoder
#enc.categorical_features()
ohefeatures = []
for x in AuthorTextCopy:
    tempList = x.split(" ")
    topicReduced = [val for val in tempList if val in label_encoder_list]
    if topicReduced != '':
        ohefeatures.append(topicReduced)

In [328]:
ohefeaturesFlattened = sum(ohefeatures,[])
#enc.fit(ohefeatures)
#enc_topics = enc.transform(summaryReducedTopics)

In [364]:
enc = preprocessing.LabelEncoder()
list(set(ohefeaturesFlattened))
ohefeaturesArray = list(set(ohefeaturesFlattened))

True


In [363]:
ohefeaturesArray
#enc.fit(ohefeaturesArray)
enc.fit(ohefeaturesArray)
enc.transform(ohefeatures)
#ohefeatures

ValueError: y contains new labels: [[]
 [u'10', u'doing', u'announcing', u'operating', u'200', u'million', u'monthly', u'active', u'devices', u'company', u'today', u'used', u'announce', u'number', u'million', u'monthly', u'active']
 [u'10', u'early', u'long', u'support', u'specific', u'company', u'announced', u'developers', u'conference', u'expanding', u'support', u'operating', u'developers']
 ...,
 [u'young', u'children', u'old', u'remember', u'car', u'want', u'founders', u'campaign']
 [u'young', u'man', u'named', u'soon', u'want', u'person', u'right', u'meet', u'looking', u'companies', u'pitch', u'like', u'tell', u'world']
 [u'young', u'startup', u'founders', u'startup', u'ways', u'half', u'period']]

In [372]:
datasetDf['author_actor_id'] = datasetDf.index
X = datasetDf[['author_actor_id','date_difference','AuthorEventsRatio']]
X = X.values.tolist()
X = np.array(X)
print X

[[  5.29000000e+02   1.00000000e+00   6.89073142e-02]
 [  5.29000000e+02   1.00000000e+00   6.89073142e-02]
 [  5.29000000e+02   0.00000000e+00   6.89073142e-02]
 ..., 
 [  8.74411100e+06   0.00000000e+00   2.78996400e-05]
 [  7.58475500e+06   0.00000000e+00   2.05450071e-05]
 [  8.07439000e+06   0.00000000e+00   6.15550796e-06]]


/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [373]:
from sklearn.cross_validation import train_test_split
import numpy as np
y = datasetDf['events']
y = y.values.tolist()
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [381]:
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.grid_search import GridSearchCV
svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)})
kr = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1), cv=5,
                  param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
                              "gamma": np.logspace(-2, 2, 5)})
svr.fit(X_train, y_train)
kr.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=0.1, kernel='rbf',
      kernel_params=None),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [1.0, 0.1, 0.01, 0.001], 'gamma': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [382]:
y_pred_svr = svr.predict(X_test)
y_pred_kr = kr.predict(X_test)
from sklearn import metrics
print metrics.mean_squared_error(y_test,y_pred_svr)
print metrics.mean_squared_error(y_test,y_pred_kr)

4438677.63828
4704674.44923


In [383]:
#Since svr with rbf gives better performance we save the svr model
joblib.dump(svr,"svr.pk")

['svr.pk',
 'svr.pk_01.npy',
 'svr.pk_02.npy',
 'svr.pk_03.npy',
 'svr.pk_04.npy',
 'svr.pk_05.npy',
 'svr.pk_06.npy',
 'svr.pk_07.npy',
 'svr.pk_08.npy',
 'svr.pk_09.npy',
 'svr.pk_10.npy',
 'svr.pk_11.npy',
 'svr.pk_12.npy',
 'svr.pk_13.npy',
 'svr.pk_14.npy',
 'svr.pk_15.npy',
 'svr.pk_16.npy',
 'svr.pk_17.npy',
 'svr.pk_18.npy',
 'svr.pk_19.npy',
 'svr.pk_20.npy',
 'svr.pk_21.npy',
 'svr.pk_22.npy',
 'svr.pk_23.npy',
 'svr.pk_24.npy',
 'svr.pk_25.npy',
 'svr.pk_26.npy',
 'svr.pk_27.npy',
 'svr.pk_28.npy',
 'svr.pk_29.npy',
 'svr.pk_30.npy',
 'svr.pk_31.npy']